# Regression Model For Predicting Movie Box Office Gross

Group Members: Ying Wu (A20370189), Yingjuan Wu (A20326320), Sahand Zeinali (A20318383)

Project Description: 
In this project, we will explore the relationship between a movie’s theatrical revenue and other key features. We will use worldwide box-office gross (numerical) as the target variable, and use relevant information that are available prior a movie's release as input variables, including general information like number of critic reviews, duration of movie (in mins), face number in poster, genres, budget, country, content-rating, imdb score, as well as social media factors like number of director facebook likes, number of cast total facebook likes, etc. 

The objective of this project is to build a regression model to predict movie box office gross. Categorical input variables include genres, country, and content-rating; numerical input variables are number of critic reviews, duration, face number in poster, budget, imdb score, number of director facebook likes, and number of cast total facebook likes. 

In [11]:
import numpy as np 
import pandas as pd
from sklearn import linear_model 
from sklearn import preprocessing
from sklearn import model_selection
from sklearn import dummy

movies = pd.read_csv("Processed_Data.csv", header = 0)
original_headers = list(movies.columns.values)#save headers in a list

In [12]:
# Scale continuous features to have 0 mean and 1 variance

# num_critic_for_reviews
movies['num_critic_for_reviews'] = preprocessing.scale(movies['num_critic_for_reviews'])
# duration
movies['duration'] = preprocessing.scale(movies['duration']) 
# director_facebook_likes
movies['director_facebook_likes'] = preprocessing.scale(movies['director_facebook_likes']) 
# cast_total_facebook_likes
movies['cast_total_facebook_likes'] = preprocessing.scale(movies['cast_total_facebook_likes']) 
# facenumber_in_poster
movies['facenumber_in_poster'] = preprocessing.scale(movies['facenumber_in_poster']) 
# budget
movies['budget'] = preprocessing.scale(movies['budget']) 
# imdb_score
movies['imdb_score'] = preprocessing.scale(movies['imdb_score']) 

# Check the shape of data
movies.head()

,num_critic_for_reviews,duration,director_facebook_likes,gross,cast_total_facebook_likes,facenumber_in_poster,budget,imdb_score,Action,Adventure,...,PG-13,PG,G,R,Not Rated,NC-17,Approved,M,GP,X
0,1.333460,3.055853,-0.107198,309000000.0,1.966068,-0.695160,3.015063,0.678332,1.0,1.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5.844645,2.806854,6.498210,448000000.0,5.016847,-0.695160,2.434369,2.029763,1.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.745964,1.213263,-0.134314,73100000.0,-0.456334,-0.186635,2.596963,0.195678,1.0,1.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.527679,-0.380328,-0.276054,201000000.0,-0.447483,-0.186635,2.550507,1.354047,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.977915,2.259057,-0.193783,302000000.0,2.507503,0.830414,2.434369,1.064455,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Performance

We use two measures to evaluate the performance of the regression models:

1.Mean Squared Error(MSE) - measures the average of the squares of the errors. It is a risk function corresponding to the expected value of the squared error loss. We use 'neg_mean_squared_error' as scoring parameter in the cross-validation function to calculate the MSE scores.

2.R^2 Score(Coefficient of Determination) - measures how well future instances are likely to be predicted by the model. The best possible score is 1. Higher R^2 score indicates better performance of regression model. We use 'r2_score' as scoring parameter in the cross-validation function to calculate the R^2 scores.

In [13]:
#Define target variable (gross) as target, define independent variables as data
movies_array = movies.as_matrix()
target = movies_array[:, 3]
data = movies_array[:, list(range(0,3))+list(range(4,len(movies_array[0])))]

In [14]:
#Baseline performance
dum = dummy.DummyRegressor()
dum.fit(data, target)
pred_mean = np.mean(dum.predict(data))
print("Predicted mean of target:",pred_mean)
print("Actual mean of target:",np.mean(target))
MSE_array = model_selection.cross_val_score(dum, data, target, cv=10, scoring = 'neg_mean_squared_error')
MSE = np.absolute(np.mean(MSE_array))
print("Baseline MSE:",MSE)
R2_array = model_selection.cross_val_score(dum, data, target, cv=10, scoring = 'r2')
print("Baseline R^2:",np.mean(R2_array))

Predicted mean of target: 45608461.1575
Actual mean of target: 45608461.1575
Baseline MSE: 4.12674080594e+15
Baseline R^2: -1.52339286602


In [15]:
#Test difference in R2 when using different methods
reg1 = linear_model.LinearRegression(fit_intercept=False)
reg1.fit(data, target)
MSE1_array = model_selection.cross_val_score(reg1, data, target, cv=10, scoring = 'neg_mean_squared_error')
MSE1 = np.absolute(np.mean(MSE1_array))
R2_array = model_selection.cross_val_score(reg1, data, target, cv=10, scoring = 'r2')
print("R2 using cross validation:",np.mean(R2_array))
r2 = reg1.score(data, target)
print("R2 using score method:",r2)
print("Linear Regression with fit_intercept=False: MSE =",MSE1)

R2 using cross validation: -0.532474714151
R2 using score method: 0.472313086101
Linear Regression with fit_intercept=False: MSE = 2.64367407615e+15


In [16]:
##Ordinary Least Squares Regression Model

#Build model with default settings
reg = linear_model.LinearRegression()
reg.fit(data, target)
#Use 10 fold cross-validation to calculate MSE
MSE_array = model_selection.cross_val_score(reg, data, target, cv=10, scoring = 'neg_mean_squared_error')
R2_array = model_selection.cross_val_score(reg, data, target, cv=10, scoring = 'r2')
MSE = np.absolute(np.mean(MSE_array))
print("Use Mean Squared Error (MSE) to measure the performance:")
print("Linear Regression with default settings:    MSE =",MSE)

#Modify parameter fit_intercept
reg1 = linear_model.LinearRegression(fit_intercept=False)
reg1.fit(data, target)
MSE1_array = model_selection.cross_val_score(reg1, data, target, cv=10, scoring = 'neg_mean_squared_error')
MSE1 = np.absolute(np.mean(MSE1_array))
print("Linear Regression with fit_intercept=False: MSE =",MSE1)

#Modify parameter n_jobs
reg4 = linear_model.LinearRegression(n_jobs=-1)
reg4.fit(data, target)
MSE4_array = model_selection.cross_val_score(reg4, data, target, cv=10, scoring = 'neg_mean_squared_error')
MSE4 = np.absolute(np.mean(MSE4_array))
print("Linear Regression with n_jobs=-1:           MSE =",MSE4)

#Modify parameter normalize
reg2 = linear_model.LinearRegression(normalize=True)
reg2.fit(data, target)
MSE2_array = model_selection.cross_val_score(reg2, data, target, cv=10, scoring = 'neg_mean_squared_error')
MSE2 = np.absolute(np.mean(MSE2_array))
print("Linear Regression with normalize=True:      MSE =",MSE2)

#Modify parameter copy_X
reg3 = linear_model.LinearRegression(copy_X=False)
reg3.fit(data, target)
MSE3_array = model_selection.cross_val_score(reg3, data, target, cv=10, scoring = 'neg_mean_squared_error')
MSE3 = np.absolute(np.mean(MSE3_array))
print("Linear Regression with false copy_X=False:  MSE =",MSE3)


Use Mean Squared Error (MSE) to measure the performance:
Linear Regression with default settings:    MSE = 2.64367167074e+15
Linear Regression with fit_intercept=False: MSE = 2.64367407615e+15
Linear Regression with n_jobs=-1:           MSE = 2.64367167074e+15
Linear Regression with normalize=True:      MSE = 2.46248008551e+42
Linear Regression with false copy_X=False:  MSE = 1.71258839024e+35


In [17]:
# Reset target and data, make sure data is not changed
target = movies_array[:, 3]
data = movies_array[:, list(range(0,3))+list(range(4,len(movies_array[0])))]

# Ridge Regression Model

# Default settings
clf = linear_model.Ridge()
clf.fit(data, target) 
MSE_array = model_selection.cross_val_score(clf, data, target, cv=10, scoring = 'neg_mean_squared_error')
MSE = np.absolute(np.mean(MSE_array))
print("Use Mean Squared Error (MSE) and Coefficient of Determination (R^2) to measure the performance:")
print("Ridge Regression with default settings:    MSE = ",MSE," R^2 = ",clf.score(data, target) )

# alpha=0.5
clf = linear_model.Ridge(alpha=0.5)
clf.fit(data, target) 
MSE_array = model_selection.cross_val_score(clf, data, target, cv=10, scoring = 'neg_mean_squared_error')
MSE = np.absolute(np.mean(MSE_array))
print("Ridge Regression with alpha=0.5:           MSE = ",MSE," R^2 = ",clf.score(data, target) )

# fit_intercept=false
clf = linear_model.Ridge(fit_intercept=False)
clf.fit(data, target) 
MSE_array = model_selection.cross_val_score(clf, data, target, cv=10, scoring = 'neg_mean_squared_error')
MSE = np.absolute(np.mean(MSE_array))
print("Ridge Regression with fit_intercept=false: MSE = ",MSE," R^2 = ",clf.score(data, target) )

# solver='lsqr'
clf = linear_model.Ridge(solver='lsqr')
clf.fit(data, target) 
MSE_array = model_selection.cross_val_score(clf, data, target, cv=10, scoring = 'neg_mean_squared_error')
MSE = np.absolute(np.mean(MSE_array))
print("Ridge Regression with solver='lsqr':       MSE = ",MSE," R^2 = ",clf.score(data, target) )


Use Mean Squared Error (MSE) and Coefficient of Determination (R^2) to measure the performance:
Ridge Regression with default settings:    MSE =  2.46586966196e+15  R^2 =  0.47149245887
Ridge Regression with alpha=0.5:           MSE =  2.49697052704e+15  R^2 =  0.471969734855
Ridge Regression with fit_intercept=false: MSE =  2.46569305504e+15  R^2 =  0.471459488646
Ridge Regression with solver='lsqr':       MSE =  2.44553236529e+15  R^2 =  0.46990451242


In [18]:
# Reset target and data, make sure data is not changed
target = movies_array[:, 3]
data = movies_array[:, list(range(0,3))+list(range(4,len(movies_array[0])))]

# Bayesian Ridge Regression Model

# Default settings
clf = linear_model.BayesianRidge()
clf.fit(data, target) 
MSE_array = model_selection.cross_val_score(clf, data, target, cv=10, scoring = 'neg_mean_squared_error')
MSE = np.absolute(np.mean(MSE_array))
print("Use Mean Squared Error (MSE) and Coefficient of Determination (R^2) to measure the performance:")
print("Bayesian Ridge Regression with default settings:           MSE = ",MSE," R^2 = ",clf.score(data, target) )

# Different alpha and lambda
clf = linear_model.BayesianRidge(alpha_1=1.e1, alpha_2=1.e2, lambda_1=1.e3, lambda_2=1.e4)
clf.fit(data, target) 
MSE_array = model_selection.cross_val_score(clf, data, target, cv=10, scoring = 'neg_mean_squared_error')
MSE = np.absolute(np.mean(MSE_array))
print("Bayesian Ridge Regression with Different alpha and lambda: MSE = ",MSE," R^2 = ",clf.score(data, target) )

# fit_intercept
clf = linear_model.BayesianRidge(fit_intercept=False)
clf.fit(data, target) 
MSE_array = model_selection.cross_val_score(clf, data, target, cv=10, scoring = 'neg_mean_squared_error')
MSE = np.absolute(np.mean(MSE_array))
print("Bayesian Ridge Regression with fit_intercept=False:        MSE = ",MSE," R^2 = ",clf.score(data, target) )

# compute_score
clf = linear_model.BayesianRidge(compute_score =True)
clf.fit(data, target) 
MSE_array = model_selection.cross_val_score(clf, data, target, cv=10, scoring = 'neg_mean_squared_error')
MSE = np.absolute(np.mean(MSE_array))
print("Bayesian Ridge Regression with compute_score =True:        MSE = ",MSE," R^2 = ",clf.score(data, target) )


Use Mean Squared Error (MSE) and Coefficient of Determination (R^2) to measure the performance:
Bayesian Ridge Regression with default settings:           MSE =  4.12674080594e+15  R^2 =  9.30033827728e-13
Bayesian Ridge Regression with Different alpha and lambda: MSE =  4.12674080591e+15  R^2 =  9.350520358e-12
Bayesian Ridge Regression with fit_intercept=False:        MSE =  5.87417706621e+15  R^2 =  -0.547289862272
Bayesian Ridge Regression with compute_score =True:        MSE =  4.12674080594e+15  R^2 =  9.30033827728e-13


In [19]:
# After reviewing all the models, we found Ridge regression model performs the best
# Therefore, we choose Ridge regression model and report the feature importance.

# Using the default settings of Ridge regression
target = movies_array[:, 3]
data = movies_array[:, list(range(0,3))+list(range(4,len(movies_array[0])))]
clf = linear_model.Ridge()
clf.fit(data, target) 

# Report Important Features
# Align feature with its name
data_name = list(movies.columns.values)
del data_name[3]
features = zip(clf.coef_, data_name)

# Sort features according to the absolute value of their weights
features = sorted(features, key = lambda x:-np.abs(x[0]))

# Print out the top ten features
for i in range(0,10):
    print("Feature #",i+1,"   ",features[i][1]," (Weight:",features[i][0],")")


Feature # 1     Hungary  (Weight: -60320344.5318 )
Feature # 2     Japan  (Weight: -49984779.1436 )
Feature # 3     West Germany  (Weight: -49962238.3826 )
Feature # 4     Peru  (Weight: 40620076.9632 )
Feature # 5     Indonesia  (Weight: -40195962.3006 )
Feature # 6     USA  (Weight: 29858289.8883 )
Feature # 7     Documentary  (Weight: -21006911.3306 )
Feature # 8     Thailand  (Weight: -20734313.7771 )
Feature # 9     num_critic_for_reviews  (Weight: 20041185.9483 )
Feature # 10     Animation  (Weight: 19504234.8581 )


In [41]:
# Reset target and data, make sure data is not changed
target = movies_array[:, 3]
data = movies_array[:, list(range(0,3))+list(range(4,len(movies_array[0])))]

# Lasso Regression Model

# Default settings
clf = linear_model.Lasso()
clf.fit(data, target) 
MSE_array = model_selection.cross_val_score(clf, data, target, cv=10, scoring = 'neg_mean_squared_error')
MSE = np.absolute(np.mean(MSE_array))
print("Use Mean Squared Error (MSE) and Coefficient of Determination (R^2) to measure the performance:")
print("Lasso with default settings:    MSE = ",MSE," R^2 = ",clf.score(data, target))

# tol changed
clf = linear_model.Lasso(tol=0.001)
clf.fit(data, target) 
MSE_array = model_selection.cross_val_score(clf, data, target, cv=10, scoring = 'neg_mean_squared_error')
MSE = np.absolute(np.mean(MSE_array))
print("Use Mean Squared Error (MSE) and Coefficient of Determination (R^2) to measure the performance:")
print("Lasso with tol=0.001:      MSE = ",MSE," R^2 = ",clf.score(data, target))

# alpha changed
clf = linear_model.Lasso(alpha=0.5)
clf.fit(data, target) 
MSE_array = model_selection.cross_val_score(clf, data, target, cv=10, scoring = 'neg_mean_squared_error')
MSE = np.absolute(np.mean(MSE_array))
print("Use Mean Squared Error (MSE) and Coefficient of Determination (R^2) to measure the performance:")
print("Lasso with alpha=0.5:      MSE = ",MSE," R^2 = ",clf.score(data, target))


#normalize changed
reg2 = linear_model.Lasso(normalize=True)
reg2.fit(data, target)
MSE2_array = model_selection.cross_val_score(reg2, data, target, cv=10, scoring = 'neg_mean_squared_error')
MSE2 = np.absolute(np.mean(MSE2_array))
print("Lasso with normalize=True:      MSE =",MSE2)


# fit_intercept changed
clf = linear_model.Lasso(fit_intercept=False)
clf.fit(data, target) 
MSE_array = model_selection.cross_val_score(clf, data, target, cv=10, scoring = 'neg_mean_squared_error')
MSE = np.absolute(np.mean(MSE_array))
print("Lasso with fit_intercept=False:        MSE = ",MSE," R^2 = ",clf.score(data, target) )

# precompute changed
clf = linear_model.Lasso(precompute=True)
clf.fit(data, target) 
MSE_array = model_selection.cross_val_score(clf, data, target, cv=10, scoring = 'neg_mean_squared_error')
MSE = np.absolute(np.mean(MSE_array))
print("Lasso with precomputer=True:        MSE = ",MSE," R^2 = ",clf.score(data, target) )

# positive changed
clf = linear_model.Lasso(positive=True)
clf.fit(data, target) 
MSE_array = model_selection.cross_val_score(clf, data, target, cv=10, scoring = 'neg_mean_squared_error')
MSE = np.absolute(np.mean(MSE_array))
print("Lasso with positive=True:        MSE = ",MSE," R^2 = ",clf.score(data, target) )

# warm_start changed
clf = linear_model.Lasso(warm_start=True)
clf.fit(data, target) 
MSE_array = model_selection.cross_val_score(clf, data, target, cv=10, scoring = 'neg_mean_squared_error')
MSE = np.absolute(np.mean(MSE_array))
print("Lasso with warm_start=True:        MSE = ",MSE," R^2 = ",clf.score(data, target) )

# copy_X changed
clf = linear_model.Lasso(copy_X=False)
clf.fit(data, target) 
MSE_array = model_selection.cross_val_score(clf, data, target, cv=10, scoring = 'neg_mean_squared_error')
MSE = np.absolute(np.mean(MSE_array))
print("Lasso with copy_X=False:        MSE = ",MSE," R^2 = ",clf.score(data, target) )



/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:479: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:479: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:479: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Library/Frameworks/Python.framework/Versions/3.5/lib/pyth

Use Mean Squared Error (MSE) and Coefficient of Determination (R^2) to measure the performance:
Lasso with default settings:    MSE =  2.64318505957e+15  R^2 =  0.472313086077


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:479: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:479: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:479: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Library/Frameworks/Python.framework/Versions/3.5/lib/pyth

Use Mean Squared Error (MSE) and Coefficient of Determination (R^2) to measure the performance:
Lasso with tol=1:      MSE =  2.64318504822e+15  R^2 =  0.472313086077


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:479: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:479: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:479: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Library/Frameworks/Python.framework/Versions/3.5/lib/pyth

...

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:479: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:479: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:479: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Library/Frameworks/Python.framework/Versions/3.5/lib/pyth

Lasso with warm_start=True:        MSE =  2.64318505957e+15  R^2 =  0.472313086077


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:479: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:479: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:479: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Library/Frameworks/Python.framework/Versions/3.5/lib/pyth

Lasso with copy_X=False:        MSE =  2.64318505957e+15  R^2 =  0.24029614075


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:479: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
